In [5]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
import requests
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from haversine import haversine, Unit

In [ ]:
'../../../../../home/jellybee/github/LLM_Research/LLM_Flo'

In [ ]:
'../../../'

In [6]:
key = open('../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

FileNotFoundError: [Errno 2] No such file or directory: '../../../api_key.txt'